In [109]:
path = r"/home/kchen/Documents/AL/Data/Raw/Joined"

In [4]:
import os
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_val_score
import xgboost as xgb

In [5]:
#Open csv file
data = pd.read_csv(r'/home/kchen/Documents/AL/Data/data1.csv', index_col='CASEID')
data.head()

/home/kchen/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3,4,5,6,7,8,13,14,15,38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,PRNCPTX,CPT,OTHERCPT1,OTHERCPT2,OTHERCPT3,OTHERCPT4,OTHERCPT5,OTHERCPT6,OTHERCPT7,OTHERCPT8,...,PRALBUM,PRBILI,PRALKPH,PRWBC,PRHCT,PRPLATE,PRPTT,PRINR,PRPT,COL_ANASTOMOTIC
CASEID,,,,,,,,,,,,,,,,,,,,,
8080841,COLECTOMY PRTL W/RMVL TERMINAL ILEUM & ILEOCOLOS,44160.0,44120,49203,0,0,0,0.0,0.0,0.0,...,3.0,0.2,81.0,6.70,26.9,333.0,44.3,1.03,NaN,0
8080850,COLECTOMY PRTL W/RMVL TERMINAL ILEUM & ILEOCOLOS,44160.0,49000,44005,44899,44120,0,0.0,0.0,0.0,...,2.7,0.2,46.0,3.20,26.2,192.0,37.0,2.00,NaN,0
8080874,LAPS COLECTOMY PRTL W/RMVL TERMINAL ILEUM,44205.0,0,0,0,0,0,0.0,0.0,0.0,...,3.6,0.5,82.0,10.82,35.9,360.0,30.9,1.20,NaN,0
8080921,COLECTOMY PARTIAL W/ANASTOMOSIS,44140.0,49000,0,0,0,0,0.0,0.0,0.0,...,2.3,3.3,143.0,8.03,22.0,101.0,83.5,1.30,NaN,0
8080981,COLECTOMY PRTL W/RMVL TERMINAL ILEUM & ILEOCOLOS,44160.0,47600,49203,47399,49000,0,0.0,0.0,0.0,...,4.1,0.3,188.0,10.20,42.0,238.0,NaN,NaN,NaN,0


In [8]:
features = ['PRNCPTX', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'CONCPT1', 'COL_STEROID', 'COL_MECH_BOWEL_PREP', 'COL_ORAL_ANTIBIOTIC', 'COL_CHEMO', 'COL_INDICATION', 'COL_EMERGENT', 'SEX', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'ASACLAS', 'TRANST','WNDCLAS', 'CPT', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10', 'OPERYR', 'COL_APPROACH', 'COL_ICD9_INDICATION', 'COL_ANASTOMOTIC', 'AGE', 'HEIGHT', 'WEIGHT', 'BMI', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRWBC', 'PRHCT', 'PRPLATE']
data = data[features]
data.shape

(146781, 65)

In [12]:
# Assign categorical and numerical columns and cast cat as cat
cat_cols = ['PRNCPTX', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'CONCPT1', 'COL_STEROID', 'COL_MECH_BOWEL_PREP', 'COL_ORAL_ANTIBIOTIC', 'COL_CHEMO', 'COL_INDICATION', 'COL_EMERGENT', 'SEX', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'ASACLAS', 'TRANST','WNDCLAS', 'CPT', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10', 'OPERYR', 'COL_APPROACH', 'COL_ICD9_INDICATION', 'COL_ANASTOMOTIC']
num_cols = ['AGE', 'HEIGHT', 'WEIGHT', 'BMI', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRWBC', 'PRHCT', 'PRPLATE']
for c in cat_cols:
    data[c] = data[c].astype(CategoricalDtype())

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146781 entries, 8080841 to 7813386
Data columns (total 65 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   PRNCPTX              146781 non-null  category
 1   OTHERCPT1            146781 non-null  category
 2   OTHERCPT2            146781 non-null  category
 3   OTHERCPT3            146781 non-null  category
 4   OTHERCPT4            146781 non-null  category
 5   OTHERCPT5            146781 non-null  category
 6   CONCPT1              146781 non-null  category
 7   COL_STEROID          145473 non-null  category
 8   COL_MECH_BOWEL_PREP  128593 non-null  category
 9   COL_ORAL_ANTIBIOTIC  130370 non-null  category
 10  COL_CHEMO            145551 non-null  category
 11  COL_INDICATION       146780 non-null  category
 12  COL_EMERGENT         13137 non-null   category
 13  SEX                  146781 non-null  category
 14  RACE_NEW             146781 non-null  categor

In [14]:
# Apply label encoder to each column with categorical data
data_label = data.copy()

label_encoder = LabelEncoder()
for col in cat_cols:
    data_label[col] = label_encoder.fit_transform(data[col].astype(str))
data_label.head(10)


,PRNCPTX,OTHERCPT1,OTHERCPT2,OTHERCPT3,OTHERCPT4,OTHERCPT5,CONCPT1,COL_STEROID,COL_MECH_BOWEL_PREP,COL_ORAL_ANTIBIOTIC,...,HEIGHT,WEIGHT,BMI,PRSODM,PRBUN,PRCREAT,PRALBUM,PRWBC,PRHCT,PRPLATE
CASEID,,,,,,,,,,,,,,,,,,,,,
8080841,4,319,566,0,0,0,0,0,0,0,...,1.6764,70.760352,25.178780,141.0,13.0,0.59,3.0,6.70,26.9,333.0
8080850,4,616,250,270,133,0,0,0,0,0,...,1.7780,78.471416,24.822640,137.0,20.0,0.98,2.7,3.20,26.2,192.0
8080874,8,0,0,0,0,0,0,2,0,2,...,1.5494,89.357624,37.222412,140.0,7.0,0.51,3.6,10.82,35.9,360.0
8080921,0,616,0,0,0,0,0,0,0,0,...,1.8288,89.811216,26.853330,138.0,11.0,1.02,2.3,8.03,22.0,101.0
8080981,4,587,566,370,251,0,0,0,0,2,...,1.6764,68.945984,24.533170,143.0,12.0,0.80,4.1,10.20,42.0,238.0
8081002,7,359,0,0,0,0,0,0,2,3,...,1.8034,106.594120,32.775484,NaN,NaN,NaN,NaN,11.60,43.6,241.0
8081172,8,0,0,0,0,0,0,0,2,2,...,1.7526,83.914520,27.319422,144.0,16.0,1.04,3.4,5.60,33.9,253.0
8081213,8,0,0,0,0,0,0,0,0,0,...,1.7018,69.853168,24.119543,139.0,34.0,1.43,3.2,9.70,27.5,227.0
8081252,8,0,0,0,0,0,0,0,2,2,...,1.7018,64.863656,22.396718,135.0,7.0,0.51,4.0,9.08,34.7,365.0


In [15]:
data_imputed = data_label.copy()

In [16]:
cat_cols = ['PRNCPTX', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'CONCPT1', 'COL_STEROID', 'COL_MECH_BOWEL_PREP', 'COL_ORAL_ANTIBIOTIC', 'COL_CHEMO', 'COL_INDICATION', 'SEX', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'ASACLAS', 'TRANST','WNDCLAS', 'CPT', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10', 'OPERYR', 'COL_APPROACH', 'COL_ANASTOMOTIC']
num_cols = ['AGE', 'HEIGHT', 'WEIGHT', 'BMI', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRWBC', 'PRHCT', 'PRPLATE']
static_cols = ['COL_EMERGENT', 'COL_ICD9_INDICATION']

In [17]:
data_imputed = data.copy()

In [21]:
cat_imputer = SimpleImputer(strategy = 'most_frequent')
num_imputer = SimpleImputer(strategy = 'median')
s_imputer = SimpleImputer(strategy='constant', fill_value=0)

# Imputation for categorical:
for col in cat_cols:
    data_imputed[col] = cat_imputer.fit_transform(data_label[col].values.reshape(-1, 1))
# Imputation for numeric:
for ncol in num_cols:
    data_imputed[ncol] = num_imputer.fit_transform(data_label[ncol].values.reshape(-1, 1))
for scol in static_cols:
    data_imputed[scol] = s_imputer.fit_transform(data_label[scol].values.reshape(-1, 1))

# Imputation removed column names; put them back
data_imputed.columns = data_label.columns

In [20]:
data_label['COL_EMERGENT'].value_counts()

7    133644
1      4993
4      3917
2      1576
3      1121
0       830
5       486
6       214
Name: COL_EMERGENT, dtype: int64

In [17]:
data['COL_EMERGENT'].value_counts(dropna=False)

NaN                                                                             133644
Obstruction                                                                       4993
Perforation                                                                       3917
Other (enter ICD-10 code)                                                         1576
Other (enter ICD-9 code)                                                          1121
Bleeding                                                                           830
Toxic colitis (Toxic Megacolon, C. diff w/out perforation, Ischemic Colitis)       486
Unknown                                                                            214
Name: COL_EMERGENT, dtype: int64

In [35]:
features = ['PRNCPTX', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'CONCPT1', 'COL_STEROID', 'COL_MECH_BOWEL_PREP', 'COL_ORAL_ANTIBIOTIC', 'COL_CHEMO', 'COL_INDICATION', 'COL_EMERGENT', 'SEX', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'ASACLAS', 'TRANST','WNDCLAS', 'CPT', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10', 'OPERYR', 'COL_APPROACH', 'AGE', 'HEIGHT', 'WEIGHT', 'BMI', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRWBC', 'PRHCT', 'PRPLATE', 'COL_ANASTOMOTIC']

In [23]:
data_imputed[num_cols].describe()

,AGE,HEIGHT,WEIGHT,BMI,PRSODM,PRBUN,PRCREAT,PRALBUM,PRWBC,PRHCT,PRPLATE
count,146781.000000,146781.000000,146781.000000,146781.000000,146781.000000,146781.000000,146781.000000,146781.000000,146781.000000,146781.000000,146781.000000
mean,61.991089,1.675640,80.695720,28.634822,139.109582,15.256969,0.955597,3.835439,7.887290,38.316687,265.744685
std,14.931203,0.104539,21.221249,6.665390,3.035788,8.921013,0.627398,0.525534,3.516003,5.721455,91.876614
min,18.000000,1.016000,14.061352,9.097328,104.000000,1.000000,0.100000,1.000000,0.100000,8.000000,1.070000
25%,52.000000,1.600200,65.770840,24.119543,138.000000,11.000000,0.720000,3.700000,5.900000,35.000000,209.000000
50%,63.000000,1.676400,78.471416,27.635281,139.000000,14.000000,0.859700,3.900000,7.200000,39.000000,252.000000
75%,73.000000,1.752600,92.532768,32.007564,141.000000,17.000000,1.000000,4.100000,8.900000,42.200000,306.000000
max,91.000000,2.184400,271.248016,107.311955,170.000000,200.000000,15.000000,9.500000,106.200000,60.000000,993.000000


In [18]:
data.shape

(146781, 63)

In [24]:
data_imputed.to_csv(r'/home/kchen/Documents/AL/Data/data2.csv', index=True, index_label='CASEID')

In [20]:
data.head(10)

,PRNCPTX,OTHERCPT1,OTHERCPT2,OTHERCPT3,OTHERCPT4,OTHERCPT5,CONCPT1,COL_STEROID,COL_MECH_BOWEL_PREP,COL_ORAL_ANTIBIOTIC,...,HEIGHT,WEIGHT,BMI,PRSODM,PRBUN,PRCREAT,PRALBUM,PRWBC,PRHCT,PRPLATE
CASEID,,,,,,,,,,,,,,,,,,,,,
8080841,4,319,566,0,0,0,0,0,0,0,...,1.6764,70.760352,25.178780,141.0,13.0,0.5900,3.0,6.70,26.9,333.0
8080850,4,616,250,270,133,0,0,0,0,0,...,1.7780,78.471416,24.822640,137.0,20.0,0.9800,2.7,3.20,26.2,192.0
8080874,8,0,0,0,0,0,0,2,0,2,...,1.5494,89.357624,37.222412,140.0,7.0,0.5100,3.6,10.82,35.9,360.0
8080921,0,616,0,0,0,0,0,0,0,0,...,1.8288,89.811216,26.853330,138.0,11.0,1.0200,2.3,8.03,22.0,101.0
8080981,4,587,566,370,251,0,0,0,0,2,...,1.6764,68.945984,24.533170,143.0,12.0,0.8000,4.1,10.20,42.0,238.0
8081002,7,359,0,0,0,0,0,0,2,3,...,1.8034,106.594120,32.775484,139.0,14.0,0.8597,3.9,11.60,43.6,241.0
8081172,8,0,0,0,0,0,0,0,2,2,...,1.7526,83.914520,27.319422,144.0,16.0,1.0400,3.4,5.60,33.9,253.0
8081213,8,0,0,0,0,0,0,0,0,0,...,1.7018,69.853168,24.119543,139.0,34.0,1.4300,3.2,9.70,27.5,227.0
8081252,8,0,0,0,0,0,0,0,2,2,...,1.7018,64.863656,22.396718,135.0,7.0,0.5100,4.0,9.08,34.7,365.0


In [22]:
y = data_imputed['COL_ANASTOMOTIC']
X = data[features]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [38]:
model1 = RandomForestClassifier(n_estimators=1000, min_samples_leaf=5, random_state=0)
model1.fit(X_train, y_train)
preds = model1.predict(X_valid)
score = roc_auc_score(y_valid, preds)
print('AUC:', score)

AUC: 0.5


In [50]:
model2 = XGBClassifier(n_estimators=60, use_label_encoder=False, n_jobs=6, random_state=0)
model2.fit(X_train, y_train)
preds2 = model2.predict(X_valid)
score2 = roc_auc_score(y_valid, preds2)
print('AUC:', score2)

[12:30:20] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
AUC: 0.4999298048575039


In [40]:
scores = cross_val_score(model1, X, y, cv=5, scoring='roc_auc', n_jobs=6)
print(scores)
print((scores.sum()) / 5)

[0.62925355 0.64644642 0.65870013 0.62748748 0.62978266]
0.6383340490847028


In [51]:
scores2 = cross_val_score(model2, X, y, cv=5, scoring='roc_auc', n_jobs=6)
print(scores2)
print((scores2.sum()) / 5)

[12:31:06] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:31:06] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:31:06] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:31:07] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [52]:
D_train = xgb.DMatrix(X_train, label=y_train, enable_categorical=False)
D_test = xgb.DMatrix(X_valid, label=y_valid, enable_categorical=False)

In [44]:
X_train.head()

,PRNCPTX,OTHERCPT1,OTHERCPT2,OTHERCPT3,OTHERCPT4,OTHERCPT5,CONCPT1,COL_STEROID,COL_MECH_BOWEL_PREP,COL_ORAL_ANTIBIOTIC,...,HEIGHT,WEIGHT,BMI,PRSODM,PRBUN,PRCREAT,PRALBUM,PRWBC,PRHCT,PRPLATE
CASEID,,,,,,,,,,,,,,,,,,,,,
8999718,3,359,624,0,0,0,0,0,2,2,...,1.7526,74.842680,24.365971,140.0,23.000,0.7700,4.1,8.55,45.8,374.0
3309884,6,0,0,0,0,0,0,0,0,2,...,1.6256,61.234920,23.172440,139.0,14.000,0.8597,3.9,7.20,39.0,252.0
8886693,6,359,407,0,0,0,0,0,2,2,...,1.6764,74.842680,26.631402,140.0,19.608,0.8400,3.9,11.00,39.0,218.0
4221844,4,0,0,0,0,0,0,0,0,0,...,1.7018,70.306760,24.276163,140.0,52.000,2.5600,2.4,6.90,36.0,456.0
3679718,6,359,0,0,0,0,0,0,0,0,...,1.8542,119.748288,34.830215,134.0,12.000,1.2000,4.4,8.80,48.7,287.0


In [56]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'binary:logistic',  
     } 

steps = 60
model3 = xgb.train(param, D_train, steps)
preds = model3.predict(D_test)
score = roc_auc_score(y_valid, preds)
print(score)

[13:36:38] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.6616213333181852


In [46]:
D_train2 = xgb.DMatrix(X, label=y, enable_categorical=False)

In [48]:
model4 = xgb.cv(param, D_train2, num_boost_round=100, nfold=5, metrics={'auc'}, seed=0, callbacks=[xgb.callback.EarlyStopping(3)])
print(model4)

    train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0         0.607065       0.002982       0.599754      0.008621
1         0.624211       0.003521       0.614025      0.005970
2         0.627817       0.001998       0.618210      0.007720
3         0.630933       0.001155       0.620451      0.008139
4         0.634090       0.002571       0.622546      0.006476
5         0.635098       0.002084       0.623116      0.006911
6         0.637236       0.002455       0.624912      0.006679
7         0.639701       0.001936       0.626301      0.008077
8         0.641924       0.001809       0.627014      0.008600
9         0.645117       0.002003       0.630104      0.009253
10        0.648419       0.002376       0.632412      0.008335
11        0.651990       0.002563       0.635291      0.007514
12        0.656479       0.002380       0.637487      0.006963
13        0.659870       0.001645       0.641522      0.008075
14        0.662607       0.001696       0.642357      0

In [23]:
data = pd.read_csv(r'/home/kchen/Documents/AL/Data/data6.csv', index_col=0)
data.head()

,PRNCPTX,CPT,OTHERCPT1,OTHERCPT2,OTHERCPT3,OTHERCPT4,OTHERCPT5,OTHERCPT6,OTHERCPT7,OTHERCPT8,...,TRANSFUS,PRSEPIS,PRSODM,PRBUN,PRCREAT,PRALBUM,PRWBC,PRHCT,PRPLATE,COL_ANASTOMOTIC
0,4.0,3.0,372.0,693.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,141.0,13.0,0.59,3.0,6.70,26.9,333.0,0.0
1,4.0,3.0,760.0,307.0,375.0,179.0,0.0,1.0,0.0,0.0,...,0.0,1.0,137.0,20.0,0.98,2.7,3.20,26.2,192.0,0.0
2,8.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,140.0,7.0,0.51,3.6,10.82,35.9,360.0,0.0
3,0.0,0.0,760.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,3.0,138.0,11.0,1.02,2.3,8.03,22.0,101.0,0.0
4,4.0,3.0,729.0,693.0,510.0,367.0,0.0,1.0,0.0,0.0,...,0.0,0.0,143.0,12.0,0.80,4.1,10.20,42.0,238.0,0.0
